In [13]:
!pip install mysql-connector-python xmltodict pandas

In [44]:
import xmltodict
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Configuración de conexión a la base de datos
DB_CONFIG = {
    'host': 'localhost',  # Cambia esto si tu base de datos está en un contenedor
    'user': 'root',
    'password': 'root123',  # Usa tu contraseña
    'database': 'realestatedb'  # Nombre de la base de datos actualizado
}

# Función para leer y procesar el archivo XML
def read_and_process_xml(file_path):
    print(f"Leyendo archivo XML: {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as file:
        data = xmltodict.parse(file.read())

    # Imprimir la estructura del XML para depurar
    print(data)

    # Extraer datos relevantes
    users = []
    properties = []

    # Iterar sobre las propiedades en el archivo XML
    for item in data['data']['listing']:
        # Propiedad
        properties.append({
            'estado': item['state'],
            'ciudad': item['city'],
            'colonia': item['colony'],
            'calle': item['street'],
            'numero_exterior': item['external_num'] if item['external_num'] != 'null' else None,
            'tipo_inmueble': item['type'],
            'transaccion': item['purpose'],
            'precio': float(item['price']),
            'codigo_proveedor': item['code'],
            'correo_contacto': item['mail_contact'],
            'telefono_contacto': item['phone_contact'],
        })

        # Usuario (correo de contacto)
        users.append({'correo_contacto': item['mail_contact']})

    # Convertir listas a DataFrames
    df_users = pd.DataFrame(users).drop_duplicates()
    df_properties = pd.DataFrame(properties)

    print("Datos procesados:")
    print(f"Usuarios: {len(df_users)}")
    print(f"Propiedades: {len(df_properties)}")

    return df_users, df_properties

In [45]:
# Función para cargar datos en la base de datos MySQL
def load_to_mysql(df_users, df_properties):
    print("Cargando datos en la base de datos MySQL...")
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        if connection.is_connected():
            cursor = connection.cursor()

            # Insertar usuarios
            for _, row in df_users.iterrows():
                cursor.execute("""
                    INSERT IGNORE INTO usuarios (correo_contacto)
                    VALUES (%s)
                """, (row['correo_contacto'],))

            # Insertar propiedades
            for _, row in df_properties.iterrows():
                cursor.execute("""
                    INSERT INTO propiedades (
                        estado, ciudad, colonia, calle, numero_exterior,
                        tipo_inmueble, transaccion, precio, codigo_proveedor,
                        correo_contacto, telefono_contacto
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    row['estado'], row['ciudad'], row['colonia'], row['calle'],
                    row['numero_exterior'], row['tipo_inmueble'],
                    row['transaccion'], row['precio'], row['codigo_proveedor'],
                    row['correo_contacto'], row['telefono_contacto']
                ))

            # Confirmar los cambios
            connection.commit()
            print("Datos cargados exitosamente en la base de datos.")
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [46]:
# Ruta al archivo XML (ajustada para Windows)
file_path = r'C:\Users\Estudiante\Documents\Proyectohubi\feed.xml'  # Ruta del archivo

# Iniciar el proceso ETL
print("Iniciando proceso ETL...")
df_users, df_properties = read_and_process_xml(file_path)

# Cargar los datos a MySQL
print("Cargando datos a MySQL...")
load_to_mysql(df_users, df_properties)

print("Proceso ETL completado.")

Iniciando proceso ETL...
Leyendo archivo XML: C:\Users\Estudiante\Documents\Proyectohubi\feed.xml...


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Datos procesados:
Usuarios: 9481
Propiedades: 30000
Cargando datos a MySQL...
Cargando datos en la base de datos MySQL...
Datos cargados exitosamente en la base de datos.
Proceso ETL completado.
